In [1]:
import sys
sys.path.insert(0,"/home2/tathagato/summarization/MACSUM/naacl")
import os
from eval import evaluate
import pickle as pkl
import tqdm
import re


[nltk_data] Downloading package punkt to /home2/tathagato/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home2/tathagato/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home2/tathagato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home2/tathagato/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


loading brown clusters...
done; # words:  247339
loading word embeddings...
done; # words:  268810


In [2]:
attributes = ['specificity', 'length']
arguments = [
    '/scratch/tathagato/naacl/zero_shot/llama_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/zero_shot/mistral_length_and_specificity.pkl',

    '/scratch/tathagato/naacl/joint_mult_attribute_sft/llama_length_and_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/joint_mult_attribute_sft/mistral_length_and_specificity/model_length_and_specificity_results_length_and_specificity.pkl',

    '/scratch/tathagato/naacl/joint_mult_attribute_dpo/llama_length_and_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/joint_mult_attribute_dpo/mistral_length_and_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    
    
    '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/llama_specificity_then_length/model_specificity_and_length_results_specificity_and_length.pkl',
    '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/llama_length_then_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/mistral_length_then_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/mistral_specificity_then_length/model_specificity_and_length_results_specificity_and_length.pkl',

    '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_dpo/llama_specificity_then_length/model_specificity_and_length_results_specificity_and_length.pkl',
    '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_dpo/llama_length_then_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_dpo/mistral_length_then_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_dpo/mistral_specificity_then_length/model_specificity_and_length_results_specificity_and_length.pkl',
    
    '/scratch/tathagato/naacl/multi_attribute_multi_adapter_dpo/llama_specificity_then_length/model_specificity_and_length_results_specificity_and_length.pkl',
    '/scratch/tathagato/naacl/multi_attribute_multi_adapter_dpo/llama_length_then_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/multi_attribute_multi_adapter_dpo/mistral_length_then_specificity/model_length_and_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/multi_attribute_multi_adapter_dpo/mistral_specificity_then_length/model_specificity_and_length_results_specificity_and_length.pkl',

    '/scratch/tathagato/naacl/multi_attribute_multi_adapter_sft/llama_specificity_then_length/model_specificity_then_length_results_specificity_and_length.pkl',
    '/scratch/tathagato/naacl/multi_attribute_multi_adapter_sft/llama_length_then_specificity/model_length_then_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/multi_attribute_multi_adapter_sft/mistral_length_then_specificity/model_length_then_specificity_results_length_and_specificity.pkl',
    '/scratch/tathagato/naacl/multi_attribute_multi_adapter_sft/mistral_specificity_then_length/model_specificity_then_length_results_specificity_and_length.pkl',



]



In [3]:
#check if the files exist
for arg in arguments:
    assert os.path.exists(arg), f"{arg} does not exist"
    

In [15]:

def extract_floats(string):
    # This pattern matches numbers with optional negative sign,
    # decimal point, and exponent notation
    pattern = r'-?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?'
    
    # Find all matches in the string
    matches = re.findall(pattern, string)
    
    # Convert matches to float
    floats = [float(match) for match in matches]
    
    return floats
def extract_words_around_then(filepath):
    # Check if "then" is in the filepath
    if "then" not in filepath:
        return ""
    
    # Regular expression to match the pattern
    pattern = r'_([^_]+)_then_([^_]+)_'
    
    # Search for the pattern in the filepath
    match = re.search(pattern, filepath)
    
    if match:
        # Extract the words before and after "then"
        word_before = match.group(1)
        word_after = match.group(2)
        return f"{word_before[0]}->{word_after[0]}"

def extract_meta_information(path):
    config = ""
    model = "Llama-3.1-Storm-8B"
    if "mistral" in path:
        model = "Mistral-7b-v3"
    method = "not sure"
    if "zero_shot" in path:
        method = "zero shot"

    if "adapter_fusion" in path:
        method = "adapter fusion"
        w1, w2 = extract_floats(path)[1], extract_floats(path)[2]
        config = f"{w1}, {w2}"

    elif "joint_mult_attribute" in path:
        method = "joint training"
    elif "multi_attribute_single_adapter_continued" in path:
        method = "single adapter continous"
        config = extract_words_around_then(path)
    elif "multi_attribute_multi_adapter" in path:
        method = "multi adapter"
        config = extract_words_around_then(path)
    
    objective = ""
    if "sft" in path:
        objective = "SFT"
    elif "dpo" in path:
        objective = "DPO"
    return model, method, objective, config

def display_length_and_specificity(res):
    specificity_keys = ['normal', 'high']
    specificity_metrics = ['specificity']
    length_keys = ['short', 'normal', 'long']
    length_metrics = ['summary_length', 'compression_ratio']
    length_res = res['result']['length']
    specificity_res = res['result']['specificity']
    specificity_outputs = []
    length_outputs = []
    model = res['model']
    method = res['method']
    objective = res['objective']
    
    # Collect metrics
    for key in specificity_keys:
        key_outputs = []
        for met in specificity_metrics:
            key_outputs.append(round(specificity_res[key][met], 2))
        specificity_outputs.append(key_outputs)
            
    for key in length_keys:
        key_outputs = []
        for met in length_metrics:
            key_outputs.append(round(length_res[key][met], 2))
        length_outputs.append(key_outputs)

    #print(extractive_outputs)
    #print(length_outputs)
    # Print in LaTeX format
    # print(f"\\multirow{{4}}{{*}}{{{method}}} & \\multirow{{4}}{{*}}{{{objective}}} & \\multirow{{4}}{{*}}{{}} & Length & {length_outputs[0][0]} & {length_outputs[1][0]} & {length_outputs[2][0]} & Density & {extractive_outputs[0][0]} & {extractive_outputs[1][0]} & {extractive_outputs[2][0]} \\\\")
    # print(f"& & & Compression Ratio & {length_outputs[0][1]} & {length_outputs[1][1]} & {length_outputs[2][1]} & Coverage & {extractive_outputs
    # print(f"& & & & & & & Overlap & {extractive_outputs[0][2]} & {extractive_outputs[1][2]} & {extractive_outputs[2][2]} \\\\")
    # print(f"& & & & & & & Rouge-P & {extractive_outputs[0][3]} & {extractive_outputs[1][3]} & {extractive_outputs[2][3]} \\\\")
    # print("\\hline")

    print(f"{method} & {objective} & {config} & {length_outputs[0][0]}/{length_outputs[1][0]}/{length_outputs[2][0]} & {length_outputs[0][1]}/{length_outputs[1][1]}/{length_outputs[2][1]} & {specificity_outputs[0][0]}/{specificity_outputs[1][0]}  \\\\")


    


In [17]:
final_res = {}
for file in tqdm.tqdm(arguments):
    model , method, objective, config = extract_meta_information(file)
    try:
        res = evaluate(file, model=model, attributes=attributes)
        final_res[file] = {"result": res, "model": model, "method": method, "objective": objective, "config": config}
    except Exception as e:
        print(f"Error evaluating {file}: {e}")
        res = None


  0%|          | 0/22 [00:00<?, ?it/s]

evaluating Llama-3.1-Storm-8B with attributes ['specificity', 'length']
length of data is  547
/scratch/tathagato/naacl/zero_shot/llama_length_and_specificity.pkl is being evaluated on specificity


/home2/tathagato/miniconda3/envs/roy/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Processing strings in parallel...
number of strings to process:  547
number of cpus:  64
size of specificity_scores:  547
Processing strings in parallel...
number of strings to process:  547
number of cpus:  64
size of specificity_scores:  547


specificity evaluation
--------------overall----------------
specificity : 0.5718596182429946
gold_specificity : 0.5665536802926032
rouge1 : 0.30072831872611555
rouge2 : 0.09671222833880148
rouge3 : 0.04060400650635695
rougeL : 0.18664877959818812
number : 547
-------------------------------------------------
--------------normal----------------
specificity : 0.5767057906273733
gold_specificity : 0.5682748626435224
rouge1 : 0.31066040502507686
rouge2 : 0.10032309291962971
rouge3 : 0.041984337808153506
rougeL : 0.1915524511754376
number : 475
-------------------------------------------------
--------------high----------------
specificity : 0.5398883420960521
gold_specificity : 0.5551986578386224
rouge1 : 0.23520413828157918
rouge2 : 0.0728905522

  5%|▍         | 1/22 [00:36<12:55, 36.92s/it]

...



length evaluation
--------------overall----------------
summary_length : 206.32541133455211
gold_summary_length : 55.96160877513711
compression_ratio : 0.279626018663752
gold_compression_ratio : 0.073909843888783
rouge1 : 0.261036091756642
rouge2 : 0.0951433169619721
rouge3 : 0.046775126861485994
rougeL : 0.1711999253533034
number : 547
-------------------------------------------------
--------------short----------------
summary_length : 79.272
gold_summary_length : 34.288
compression_ratio : 0.10912370131133287
gold_compression_ratio : 0.046553397518622926
rouge1 : 0.29641929196716127
rouge2 : 0.10631443263728103
rouge3 : 0.05085076001845148
rougeL : 0.20452013388819623
number : 125
-------------------------------------------------
--------------normal----------------
summary_length : 199.42662116040955
gold_summary_length : 47.89078498293515
compression_ratio : 0.23817346549504617
gold_compression_ratio : 0.06213176543009471
rouge1 : 0.23785490924186392
rouge2 : 0.08179243308550466

 95%|█████████▌| 21/22 [13:07<00:37, 37.56s/it]



length evaluation
--------------overall----------------
summary_length : 70.43327239488117
gold_summary_length : 55.96160877513711
compression_ratio : 0.09853122703875238
gold_compression_ratio : 0.073909843888783
rouge1 : 0.24436634012024583
rouge2 : 0.06343769975936132
rouge3 : 0.02264559708530955
rougeL : 0.15953057469940202
number : 547
-------------------------------------------------
--------------short----------------
summary_length : 44.744
gold_summary_length : 34.288
compression_ratio : 0.07321968952536642
gold_compression_ratio : 0.046553397518622926
rouge1 : 0.23081549780303584
rouge2 : 0.06612693500571724
rouge3 : 0.02503523748614563
rougeL : 0.16709839525652703
number : 125
-------------------------------------------------
--------------normal----------------
summary_length : 66.71672354948805
gold_summary_length : 47.89078498293515
compression_ratio : 0.08606225667755518
gold_compression_ratio : 0.06213176543009471
rouge1 : 0.23247979514595685
rouge2 : 0.05472859490963

100%|██████████| 22/22 [13:43<00:00, 37.43s/it]



length evaluation
--------------overall----------------
summary_length : 61.9908592321755
gold_summary_length : 55.96160877513711
compression_ratio : 0.08545047640480777
gold_compression_ratio : 0.073909843888783
rouge1 : 0.24142953539313078
rouge2 : 0.06547893727696438
rouge3 : 0.023623357218326677
rougeL : 0.1592296672519679
number : 547
-------------------------------------------------
--------------short----------------
summary_length : 41.216
gold_summary_length : 34.288
compression_ratio : 0.06144169953911387
gold_compression_ratio : 0.046553397518622926
rouge1 : 0.24093647466015605
rouge2 : 0.0709855091125027
rouge3 : 0.027554486637980605
rougeL : 0.17280401423111644
number : 125
-------------------------------------------------
--------------normal----------------
summary_length : 56.14334470989761
gold_summary_length : 47.89078498293515
compression_ratio : 0.07072150279920554
gold_compression_ratio : 0.06213176543009471
rouge1 : 0.22496461287672195
rouge2 : 0.056635654094080

In [19]:
for file,res in final_res.items():
    model , method, objective, config = extract_meta_information(file)
    res['model'] = model
    res['method'] = method
    res['objective'] = objective
    res['config'] = config
    if model == "Llama-3.1-Storm-8B":
        display_length_and_specificity(res)

zero shot &  &  & 53.58/110.89/218.14 & 0.08/0.15/0.34 & 0.58/0.54  \\
joint training & SFT &  & 47.59/144.92/233.43 & 0.06/0.17/0.37 & 0.5/0.52  \\
joint training & DPO &  & 51.35/119.54/285.36 & 0.08/0.16/0.44 & 0.6/0.59  \\
single adapter continous & SFT & s->l & 64.58/160.95/272.88 & 0.09/0.18/0.41 & 0.51/0.51  \\
single adapter continous & SFT & l->s & 93.25/223.67/275.93 & 0.12/0.27/0.4 & 0.51/0.52  \\
single adapter continous & DPO & s->l & 45.05/81.57/238.33 & 0.07/0.11/0.37 & 0.58/0.57  \\
single adapter continous & DPO & l->s & 45.83/118.53/360.97 & 0.07/0.16/0.63 & 0.62/0.6  \\
multi adapter & DPO & s->l & 43.01/79.1/242.19 & 0.06/0.1/0.38 & 0.6/0.57  \\
multi adapter & DPO & l->s & 46.42/110.48/220.03 & 0.07/0.15/0.34 & 0.57/0.58  \\
multi adapter & SFT & s->l & 81.46/137.16/285.36 & 0.1/0.16/0.42 & 0.5/0.5  \\
multi adapter & SFT & l->s & 79.27/199.43/345.11 & 0.11/0.24/0.54 & 0.49/0.48  \\


In [20]:
for file,res in final_res.items():
    model , method, objective, config = extract_meta_information(file)
    res['model'] = model
    res['method'] = method
    res['objective'] = objective
    res['config'] = config
    if model == "Mistral-7b-v3":
        display_length_and_specificity(res)

zero shot &  &  & 168.76/212.59/259.22 & 0.27/0.28/0.4 & 0.76/0.74  \\
joint training & SFT &  & 48.51/63.52/82.26 & 0.08/0.08/0.12 & 0.7/0.66  \\
joint training & DPO &  & 156.67/206.39/248.45 & 0.26/0.28/0.39 & 0.77/0.76  \\
single adapter continous & SFT & l->s & 36.95/53.54/84.76 & 0.06/0.07/0.12 & 0.7/0.71  \\
single adapter continous & SFT & s->l & 54.22/60.96/74.35 & 0.08/0.08/0.11 & 0.67/0.64  \\
single adapter continous & DPO & l->s & 125.33/188.9/230.6 & 0.2/0.26/0.37 & 0.74/0.76  \\
single adapter continous & DPO & s->l & 161.3/205.43/234.1 & 0.26/0.28/0.37 & 0.74/0.73  \\
multi adapter & DPO & l->s & 179.32/214.69/257.98 & 0.29/0.29/0.4 & 0.77/0.77  \\
multi adapter & DPO & s->l & 150.91/193.69/240.19 & 0.25/0.26/0.38 & 0.76/0.73  \\
multi adapter & SFT & l->s & 44.74/66.72/103.77 & 0.07/0.09/0.15 & 0.69/0.7  \\
multi adapter & SFT & s->l & 41.22/56.14/95.4 & 0.06/0.07/0.14 & 0.69/0.7  \\
